# Classifying Headlines into LDA topics

## Imports

In [1]:
import pandas as pd

In [2]:
total_data = pd.read_parquet("..data/processed/news-consolidated-v2.parquet")
total_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '..data/processed/news-consolidated-v2.parquet'